## AlexNet type net

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import cv2 as cv

import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Precision, Recall, AUC, Accuracy
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
tree_types = os.listdir('../data/image data/train')
X = []
y = []
val_x = []
val_y = []

for tree in tree_types:
    tree_path = os.path.join('../data/image data/train', tree)
    tree_disease_types = os.listdir(tree_path)

    for disease in tree_disease_types:
        img_name = os.listdir(os.path.join(tree_path, disease))

        for img in img_name:
            image_path = os.path.join(tree_path, disease, img)
            image = cv.resize(cv.imread(image_path), dsize=(96, 96))
            X.append(tf.image.rgb_to_grayscale(image))
            y.append([tree, disease])

for tree in tree_types:
    tree_path = os.path.join('../data/image data/validation', tree)
    tree_disease_types = os.listdir(tree_path)

    for disease in tree_disease_types:
        img_name = os.listdir(os.path.join(tree_path, disease))

        for img in img_name:
            image_path = os.path.join(tree_path, disease, img)
            image = cv.resize(cv.imread(image_path), dsize=(96, 96))
            val_x.append(tf.image.rgb_to_grayscale(image))
            val_y.append([tree, disease])

In [ ]:

X = np.asarray(X)
val_x = np.asarray(val_x)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
val_y = mlb.transform(val_y)
train_generator = ImageDataGenerator(rescale=1. / 255)
val_generator = ImageDataGenerator(rescale=1. / 255)

In [ ]:

train_gen = train_generator.flow(X, y, batch_size=64)
val_gen = val_generator.flow(val_x, val_y, batch_size=64)

In [ ]:
check_pointer = ModelCheckpoint(filepath='checkpoints/exp_conv_3.hdf5',
                                save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss',
                           patience=15,
                           min_delta=0)

In [ ]:
exp_conv = Sequential()

exp_conv.add(InputLayer(input_shape=(96, 96, 3)))
exp_conv.add(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation=relu))
exp_conv.add(MaxPooling2D(pool_size=(2, 2)))

exp_conv.add(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation=relu))
exp_conv.add(MaxPooling2D(pool_size=(2, 2)))

exp_conv.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation=relu))
exp_conv.add(MaxPooling2D(pool_size=(2, 2)))

exp_conv.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation=relu))
exp_conv.add(MaxPooling2D(pool_size=(2, 2)))

exp_conv.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation=relu))
exp_conv.add(MaxPooling2D(pool_size=(2, 2)))

exp_conv.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation=relu))
exp_conv.add(Dropout(0.35))

exp_conv.add(Flatten())
exp_conv.add(Dense(units=512, activation=relu))
exp_conv.add(Dropout(0.4))
exp_conv.add(Dense(units=42, activation=sigmoid))

exp_conv.summary()

In [ ]:
plot_model(exp_conv,
           to_file='../models/exp_conv_3.png',
           show_shapes=True,
           show_dtype=True,
           show_layer_names=True)

In [ ]:
exp_conv.compile(optimizer=Adam(),
                 loss=binary_crossentropy,
                 metrics=[Recall(), Precision(), Accuracy()])

history = exp_conv.fit(train_gen,
                       epochs=50,
                       validation_data=val_gen,
                       callbacks=[check_pointer],
                       shuffle=True)

In [ ]:
exp_conv.evaluate(val_gen)

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=3, figsize=[18, 6], dpi=300)
axes = axes.ravel()
epochs = list(range(69))

sns.lineplot(x=epochs, y=history.history['loss'], ax=axes[0], label='loss')
sns.lineplot(x=epochs, y=history.history['val_loss'], ax=axes[0], label='val loss')
sns.lineplot(x=epochs, y=history.history['precision'], ax=axes[1], label='precision')
sns.lineplot(x=epochs, y=history.history['val_precision'], ax=axes[1], label='val precision')
sns.lineplot(x=epochs, y=history.history['recall'], color='#025918', ax=axes[1], label='recall')
sns.lineplot(x=epochs, y=history.history['val_recall'], color='#D9B504', ax=axes[1], label='val recall')
sns.lineplot(x=epochs, y=history.history['accuracy'], ax=axes[2], label='accuracy')
sns.lineplot(x=epochs, y=history.history['val_accuracy'], ax=axes[2], label='val accuracy')
axes[0].set_xlabel('epoch')
axes[0].set_ylabel('loss')
axes[1].set_xlabel('epoch')
axes[1].set_ylabel('precision and recall')
axes[2].set_xlabel('epoch')
axes[2].set_ylabel('accuracy')
plt.savefig('exp_1_1_conv_train_history.png')
plt.show()